# top-K问题

## 什么是 Top K 问题？简单来说就是在一堆数据里面找到前 K 大（当然也可以是前 K 小）的数。


### 堆排解法
### 用堆排来解决Top K的思路很直接。

### 前面已经说过，堆排利用的大（小）顶堆所有子节点元素都比父节点小（大）的性质来实现的，这里故技重施：既然一个大顶堆的顶是最大的元素，那我们要找最小的K个元素，是不是可以先建立一个包含K个元素的堆，然后遍历集合，如果集合的元素比堆顶元素小（说明它目前应该在K个最小之列），那就用该元素来替换堆顶元素，同时维护该堆的性质，那在遍历结束的时候，堆中包含的K个元素是不是就是我们要找的最小的K个元素？


### 速记口诀：最小的K个用最大堆，最大的K个用最小堆。

### 时间复杂度
### n*logK

### 速记：堆排的时间复杂度是n*logn，这里相当于只对前Top K个元素建堆排序，想法不一定对，但一定有助于记忆。

### 适用场景
### 实现的过程中，我们先用前K个数建立了一个堆，然后遍历数组来维护这个堆。这种做法带来了三个好处：（1）不会改变数据的输入顺序（按顺序读的）；（2）不会占用太多的内存空间（事实上，一次只读入一个数，内存只要求能容纳前K个数即可）；（3）由于（2），决定了它特别适合处理海量数据。

### 这三点，也决定了它最优的适用场景。

In [4]:
class Solution:
    def getLeastNumbers(self, arr, k):
        if k == 0:
            return []
        heaplist = HeapList()
        heaplist.buildHeap(arr[:k])
        for i in arr[k:]:
            if i < heaplist.heaplist[1]:
                heaplist.delMax()
                heaplist.insert(i)
        return heaplist.heaplist[1:]

class HeapList():
    """
    大顶堆
    """
    def __init__(self):
        self.heaplist = [0]
        self.size = 0

    def buildHeap(self, alist):
         i = len(alist) // 2
         self.size = len(alist)
         self.heaplist += alist[:]
         while i > 0:
             self.percDown(i)
             i -= 1

    def delMax(self):
        """删除堆顶最大元素"""
        retval = self.heaplist[1]
        self.heaplist[1] = self.heaplist[self.size]
        self.size -= 1
        self.heaplist.pop()
        self.percDown(1)
        return retval

    def insert(self, k):
        self.heaplist.append(k)
        self.size += 1
        self.percUP(self.size)

    def percUP(self, i):
        while i // 2 > 0:
            if self.heaplist[i] > self.heaplist[i // 2]:
                self.heaplist[i], self.heaplist[i // 2] = self.heaplist[i // 2], self.heaplist[i]
            i //= 2

    def percDown(self, i):
        while i * 2 <= self.size:
            mc = self.maxChild(i)
            if self.heaplist[i] < self.heaplist[mc]:
                self.heaplist[i], self.heaplist[mc] = self.heaplist[mc], self.heaplist[i]
            i = mc

    def maxChild(self, i):
        if 2 * i + 1 > self.size:
            return 2 * i
        else:
            if self.heaplist[2 * i] > self.heaplist[2 * i + 1]:
                return 2 * i
            else:
                return 2 * i + 1


## 快排解法
### 用快排的思想来解Top K问题，必然要运用到”分治”。

### 与快排相比，两者唯一的不同是在对”分治”结果的使用上。我们知道，分治函数会返回一个position，在position左边的数都比第position个数小，在position右边的数都比第position大。我们不妨不断调用分治函数，直到它输出的position = K-1，此时position前面的K个数（0到K-1）就是要找的前K个数。

### 实现：
### “分治”还是原来的那个分治，关键是getTopK的逻辑，务必要结合注释理解透彻，自动动手写写。

### 时间复杂度
### 若第k小的值出现在左侧(当中值pos - l + 1 > k)，向左递归，出现在右侧(当中值pos - l + 1 < k)，向右递归。最好O(n)，最坏会退化到O(n^2)


### 速记：记住就行，基于partition函数的时间复杂度比较难证明，从来没考过。

### 适用场景
### 对照着堆排的解法来看，partition函数会不断地交换元素的位置，所以它肯定会改变数据输入的顺序；既然要交换元素的位置，那么所有元素必须要读到内存空间中，所以它会占用比较大的空间，至少能容纳整个数组；数据越多，占用的空间必然越大，海量数据处理起来相对吃力。

### 但是，它的时间复杂度很低，意味着数据量不大时，效率极高。

In [3]:
class Solution:
    def getLeastNumbers(self, arr, k):
        def partition(arr, l, r):
            #选定中值
            pivotvalue = arr[l]
            lmark = l + 1
            rmark = r
            done = False

            while not done:
                while lmark <= rmark and arr[lmark] <= pivotvalue:
                    lmark += 1
                while rmark >= lmark and arr[rmark] >= pivotvalue:
                    rmark -= 1
                if rmark < lmark:
                    done = True
                else:
                    arr[lmark], arr[rmark] = arr[rmark], arr[lmark]

            arr[l], arr[rmark] = arr[rmark], arr[l]
            return rmark
        
        def quicksort(arr, l, r, k):
            if l > r:
                return 
            pos = partition(arr, l, r)
            num = pos - l + 1
            if k == num:
                return
            if k < num:
                quicksort(arr, l, pos - 1, k)
            else:
                quicksort(arr, pos+1, r, k - num)

        if k == 0:
            return []
        quicksort(arr, 0, len(arr) - 1, k)
        return arr[:k]


# BFPRT算法原理

## 引用： https://zhuanlan.zhihu.com/p/31498036

## 1. 将n个元素划为 n/5 组，每组5个，至多只有一组由 n mod 5 个元素组成。
## 2. 寻找这 n/5 个组中每一个组的中位数，这个过程可以用插入排序。
## 3. 对步骤2中的 n/5 个中位数，重复步骤1和步骤2，递归下去，直到剩下一个数字。
## 4. 最终剩下的数字即为pivot，把大于它的数全放左边，小于等于它的数全放右边。
## 5. 判断pivot的位置与k的大小，有选择的对左边或右边递归。

In [8]:
import random
class Solution:
    def findKthLargest(self, a, k):
        if len(a) < 5:
            a.sort(reverse=True)
            return a[k-1]
    
        total_num = len(a)
        splits = total_num//5   #一共分成这么多组
    
        split_medians = []
        for i in range(splits):
            cur = sorted(a[i*5 : (i+1)*5]) #切片操作
            mid = cur[2]
            split_medians.append(mid)
        m = self.findKthLargest(split_medians, splits//2)
        m_index = a.index(m) #记录m第一次在列表中的位置


        left, right, m_index = self.partion(a, m, m_index)
        if k == len(left)+1:
            return m
        elif k <len(left)+1:
            return self.findKthLargest(left, k)
        else:
            return self.findKthLargest(right, k-1-m_index)
        
    
    
    def partion(self,nums, m, m_index):

        nums = nums[:m_index]+nums[m_index+1:]
        if len(nums)<=1:
            return 0,0,0
        pivot = m
        left = []
        right = []
        for i in nums:
            if i >= pivot:
                left.append(i)
            else:
                right.append(i)
        return left,right, len(left)
    
sol = Solution()
 
a = [5, 3, 1, 8, 2,10, 15, 18, 11,13, 16, 17, 12, 19, 0, 6, 4, 7, 9, 14]
random.shuffle(a)
k = 3
print(sol.findKthLargest(a,7))

13


# 

# 勇者斗恶龙的题目：
## 你的王国里有一条n个头的恶龙，你希望雇佣一些骑士把它杀死（砍掉所有的头）。村里有m个骑士可以雇佣，一个能力值为x的骑士可以砍掉恶龙一个致敬不超过x的头，且需要支付x个金币。如何雇佣骑士才能砍掉恶龙的所有头，且需要支付的金币最少？注意，一个骑士只能砍一个头（且不能被雇佣两次）。

# 0-1背包问题的动态规划算法

## 参考： https://zhuanlan.zhihu.com/p/30959069

# 动态规划总结
## 参考： https://zhuanlan.zhihu.com/p/91582909


# 几个必须“背诵”的贪心算法题


In [ ]:
http://www.lintcode.com/problem/majority-number/
http://www.lintcode.com/problem/create-maximum-number/
http://www.lintcode.com/problem/jump-game-ii/
http://www.lintcode.com/problem/jump-game/
http://www.lintcode.com/problem/gas-station/
http://www.lintcode.com/problem/delete-digits/
http://www.lintcode.com/problem/task-scheduler/

# morris算法

# 拆分二叉搜索树

In [ ]:
https://zhuanlan.zhihu.com/p/89422631

# 反转二叉树  how to invert a binary tree

# 反转链表